In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed
from keras.layers import Dropout

In [2]:
import os 
os.chdir(r'C:\Users\Nikhi\Desktop\Files\ML\final\final\Market TS')

In [3]:
Xl = pd.read_csv('Xl.csv',header=None)
Xs = pd.read_csv('Xs.csv',header=None)
R = pd.read_csv('y.csv',header = None)

In [4]:
R.head()

,0
0,0.006331
1,0.006711
2,0.003352
3,0.040054
4,0.023631


In [5]:
y_agg=R.iloc[::-1].rolling(window=12).sum().iloc[::-1]

In [6]:
t_train_start = list(range(46*12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x for x in t_test_start]

In [7]:
np.shape(t_test_end)

(552,)

In [8]:
t_test_end[551]

731

In [10]:
loss = []
preds = []
seed(1)
for retrain_idx in range(552):
    X = Xl.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Xl.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],X_idx]
    X_test = Xl.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],X_idx]
    y = y_agg.iloc[t_train_start[retrain_idx]:t_train_end[retrain_idx],:]
    y_val = y_agg.loc[t_val_start[retrain_idx]:t_val_end[retrain_idx],:]
    y_test = y_agg.loc[t_test_start[retrain_idx]:t_test_end[retrain_idx],:]
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(4098,activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(LayerNormalization())
    model.add(Dense(2056,activation='sigmoid'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2056,activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1024,activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(512,activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(256,activation='sigmoid'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128,activation='tanh'))
    model.add(BatchNormalization())
    model.add(Dropout(0.05))
    model.add(Dense(64,activation='sigmoid'))
    model.add(Dense(1,activation='linear'))
    model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.0005,momentum=0.9),loss='mean_absolute_error')
    early_stop = EarlyStopping(monitor='val_loss', verbose=0, patience=30)
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=32, epochs=100,verbose=1,callbacks=[early_stop])
    preds.append(model.predict(X_test))
    loss.append(min(history.history['val_loss']))

Epoch 1/100
4/4 [==============================] - 27s 1s/step - loss: 0.8026 - val_loss: 0.1588
Epoch 2/100
4/4 [==============================] - 1s 301ms/step - loss: 0.3010 - val_loss: 0.1775
Epoch 3/100
4/4 [==============================] - 1s 197ms/step - loss: 0.2594 - val_loss: 0.3199
Epoch 4/100
4/4 [==============================] - 1s 217ms/step - loss: 0.2414 - val_loss: 0.1861
Epoch 5/100
4/4 [==============================] - 1s 221ms/step - loss: 0.1662 - val_loss: 0.1516
Epoch 6/100
4/4 [==============================] - 1s 226ms/step - loss: 0.2208 - val_loss: 0.2634
Epoch 7/100
4/4 [==============================] - 1s 269ms/step - loss: 0.1334 - val_loss: 0.1500
Epoch 8/100
4/4 [==============================] - 1s 276ms/step - loss: 0.1437 - val_loss: 0.1507
Epoch 9/100
4/4 [==============================] - 1s 310ms/step - loss: 0.1091 - val_loss: 0.2421
Epoch 10/100
4/4 [==============================] - 1s 369ms/step - loss: 0.1240 - val_loss: 0.2461
Epoch 11/10

4/4 [==============================] - 1s 306ms/step - loss: 0.1007 - val_loss: 0.1561
Epoch 24/100
4/4 [==============================] - 1s 235ms/step - loss: 0.0946 - val_loss: 0.1479
Epoch 25/100
4/4 [==============================] - 1s 276ms/step - loss: 0.1061 - val_loss: 0.1540
Epoch 26/100
4/4 [==============================] - 1s 307ms/step - loss: 0.0986 - val_loss: 0.1575
Epoch 27/100
4/4 [==============================] - 1s 360ms/step - loss: 0.0979 - val_loss: 0.1458
Epoch 28/100
4/4 [==============================] - 1s 305ms/step - loss: 0.1036 - val_loss: 0.1675
Epoch 29/100
4/4 [==============================] - 1s 262ms/step - loss: 0.1155 - val_loss: 0.1481
Epoch 30/100
4/4 [==============================] - 1s 272ms/step - loss: 0.0958 - val_loss: 0.1597
Epoch 31/100
4/4 [==============================] - 1s 341ms/step - loss: 0.0937 - val_loss: 0.1537
Epoch 32/100
4/4 [==============================] - 1s 300ms/step - loss: 0.0943 - val_loss: 0.1522
Epoch 33/100


4/4 [==============================] - 1s 235ms/step - loss: 0.1046 - val_loss: 0.1731
Epoch 54/100
4/4 [==============================] - 1s 221ms/step - loss: 0.1004 - val_loss: 0.1511
Epoch 55/100
4/4 [==============================] - 1s 241ms/step - loss: 0.1019 - val_loss: 0.1572
Epoch 56/100
4/4 [==============================] - 1s 246ms/step - loss: 0.0952 - val_loss: 0.1548
Epoch 57/100
4/4 [==============================] - 1s 176ms/step - loss: 0.0937 - val_loss: 0.1501
Epoch 58/100
4/4 [==============================] - 1s 147ms/step - loss: 0.0994 - val_loss: 0.1509
Epoch 59/100
4/4 [==============================] - 1s 152ms/step - loss: 0.0926 - val_loss: 0.1505
Epoch 60/100
4/4 [==============================] - 1s 153ms/step - loss: 0.0940 - val_loss: 0.1544
Epoch 61/100
4/4 [==============================] - 1s 145ms/step - loss: 0.0960 - val_loss: 0.1491
Epoch 62/100
4/4 [==============================] - 1s 224ms/step - loss: 0.0973 - val_loss: 0.1516
Epoch 63/100


4/4 [==============================] - 1s 231ms/step - loss: 0.1060 - val_loss: 0.1293
Epoch 41/100
4/4 [==============================] - 1s 183ms/step - loss: 0.0953 - val_loss: 0.1333
Epoch 42/100
4/4 [==============================] - 1s 217ms/step - loss: 0.1118 - val_loss: 0.1350
Epoch 43/100
4/4 [==============================] - 1s 202ms/step - loss: 0.0952 - val_loss: 0.1387
Epoch 44/100
4/4 [==============================] - 1s 223ms/step - loss: 0.0991 - val_loss: 0.1317
Epoch 45/100
4/4 [==============================] - 1s 234ms/step - loss: 0.1020 - val_loss: 0.1561
Epoch 46/100
4/4 [==============================] - 1s 189ms/step - loss: 0.1079 - val_loss: 0.1399
Epoch 47/100
4/4 [==============================] - 1s 223ms/step - loss: 0.1051 - val_loss: 0.1420
Epoch 48/100
4/4 [==============================] - 1s 202ms/step - loss: 0.1052 - val_loss: 0.1414
Epoch 49/100
4/4 [==============================] - 1s 205ms/step - loss: 0.1045 - val_loss: 0.1418
Epoch 50/100


4/4 [==============================] - 1s 280ms/step - loss: 0.2287 - val_loss: 0.3122
Epoch 3/100
4/4 [==============================] - 1s 278ms/step - loss: 0.1793 - val_loss: 0.1758
Epoch 4/100
4/4 [==============================] - 1s 254ms/step - loss: 0.1487 - val_loss: 0.1571
Epoch 5/100
4/4 [==============================] - 1s 296ms/step - loss: 0.1359 - val_loss: 0.1843
Epoch 6/100
4/4 [==============================] - 1s 214ms/step - loss: 0.1346 - val_loss: 0.1687
Epoch 7/100
4/4 [==============================] - 1s 221ms/step - loss: 0.1193 - val_loss: 0.2628
Epoch 8/100
4/4 [==============================] - 1s 264ms/step - loss: 0.1544 - val_loss: 0.1516
Epoch 9/100
4/4 [==============================] - 1s 239ms/step - loss: 0.1411 - val_loss: 0.1894
Epoch 10/100
4/4 [==============================] - 1s 252ms/step - loss: 0.1251 - val_loss: 0.1889
Epoch 11/100
4/4 [==============================] - 1s 221ms/step - loss: 0.1368 - val_loss: 0.1552
Epoch 12/100
4/4 [==

...

4/4 [==============================] - 1s 191ms/step - loss: 0.1135 - val_loss: 0.1020
Epoch 47/100
4/4 [==============================] - 1s 209ms/step - loss: 0.1100 - val_loss: 0.0699
Epoch 48/100
4/4 [==============================] - 1s 190ms/step - loss: 0.1109 - val_loss: 0.0575
Epoch 49/100
4/4 [==============================] - 1s 182ms/step - loss: 0.1137 - val_loss: 0.1157
Epoch 50/100
4/4 [==============================] - 1s 205ms/step - loss: 0.1053 - val_loss: 0.0682
Epoch 51/100
4/4 [==============================] - 1s 204ms/step - loss: 0.1201 - val_loss: 0.0744
Epoch 52/100
4/4 [==============================] - 1s 199ms/step - loss: 0.1140 - val_loss: 0.1227
Epoch 53/100
4/4 [==============================] - 1s 197ms/step - loss: 0.1164 - val_loss: 0.0684
Epoch 54/100
4/4 [==============================] - 1s 196ms/step - loss: 0.1206 - val_loss: 0.0657
Epoch 55/100
4/4 [==============================] - 1s 210ms/step - loss: 0.1121 - val_loss: 0.1268
Epoch 56/100


4/4 [==============================] - 1s 207ms/step - loss: 0.1152 - val_loss: 0.1318
Epoch 21/100
4/4 [==============================] - 1s 214ms/step - loss: 0.1067 - val_loss: 0.0584
Epoch 22/100
4/4 [==============================] - 1s 206ms/step - loss: 0.1145 - val_loss: 0.0965
Epoch 23/100
4/4 [==============================] - 1s 206ms/step - loss: 0.1211 - val_loss: 0.1384
Epoch 24/100
4/4 [==============================] - 1s 210ms/step - loss: 0.1316 - val_loss: 0.0618
Epoch 25/100
4/4 [==============================] - 1s 201ms/step - loss: 0.1198 - val_loss: 0.1402
Epoch 26/100
4/4 [==============================] - 1s 212ms/step - loss: 0.1044 - val_loss: 0.0710
Epoch 27/100
4/4 [==============================] - 1s 207ms/step - loss: 0.1209 - val_loss: 0.0590
Epoch 28/100
4/4 [==============================] - 1s 210ms/step - loss: 0.1044 - val_loss: 0.0627
Epoch 29/100
4/4 [==============================] - 1s 198ms/step - loss: 0.1219 - val_loss: 0.1147
Epoch 30/100


4/4 [==============================] - 0s 101ms/step - loss: 0.1110 - val_loss: 0.0658
Epoch 49/100
4/4 [==============================] - 0s 97ms/step - loss: 0.1145 - val_loss: 0.0811
Epoch 50/100
4/4 [==============================] - 0s 101ms/step - loss: 0.1076 - val_loss: 0.0670
Epoch 51/100
4/4 [==============================] - 0s 101ms/step - loss: 0.1133 - val_loss: 0.0772
Epoch 52/100
4/4 [==============================] - 0s 104ms/step - loss: 0.1126 - val_loss: 0.1160
Epoch 53/100
4/4 [==============================] - 0s 102ms/step - loss: 0.1014 - val_loss: 0.1020
Epoch 54/100
4/4 [==============================] - 0s 104ms/step - loss: 0.1077 - val_loss: 0.0638
Epoch 55/100
4/4 [==============================] - 0s 104ms/step - loss: 0.1043 - val_loss: 0.0635
Epoch 56/100
4/4 [==============================] - 0s 104ms/step - loss: 0.1082 - val_loss: 0.1077
Epoch 57/100
4/4 [==============================] - 0s 105ms/step - loss: 0.1073 - val_loss: 0.0809
Epoch 58/100
4

4/4 [==============================] - 0s 84ms/step - loss: 0.1074 - val_loss: 0.0741
Epoch 43/100
4/4 [==============================] - 0s 87ms/step - loss: 0.1044 - val_loss: 0.0693
Epoch 44/100
4/4 [==============================] - 0s 86ms/step - loss: 0.1035 - val_loss: 0.0826
Epoch 45/100
4/4 [==============================] - 0s 88ms/step - loss: 0.1076 - val_loss: 0.0661
Epoch 46/100
4/4 [==============================] - 0s 86ms/step - loss: 0.1035 - val_loss: 0.0893
Epoch 47/100
4/4 [==============================] - 0s 87ms/step - loss: 0.1175 - val_loss: 0.0810
Epoch 48/100
4/4 [==============================] - 0s 89ms/step - loss: 0.1047 - val_loss: 0.0627
Epoch 49/100
4/4 [==============================] - 0s 88ms/step - loss: 0.1021 - val_loss: 0.0986
Epoch 50/100
4/4 [==============================] - 0s 88ms/step - loss: 0.1050 - val_loss: 0.0664
Epoch 51/100
4/4 [==============================] - 0s 91ms/step - loss: 0.1162 - val_loss: 0.0988
Epoch 52/100
4/4 [=====

4/4 [==============================] - 0s 85ms/step - loss: 0.1798 - val_loss: 0.0620
Epoch 10/100
4/4 [==============================] - 0s 86ms/step - loss: 0.1391 - val_loss: 0.1672
Epoch 11/100
4/4 [==============================] - 0s 87ms/step - loss: 0.1489 - val_loss: 0.0664
Epoch 12/100
4/4 [==============================] - 0s 86ms/step - loss: 0.1300 - val_loss: 0.1310
Epoch 13/100
4/4 [==============================] - 0s 85ms/step - loss: 0.1415 - val_loss: 0.2237
Epoch 14/100
4/4 [==============================] - 0s 85ms/step - loss: 0.1483 - val_loss: 0.0695
Epoch 15/100
4/4 [==============================] - 0s 86ms/step - loss: 0.1374 - val_loss: 0.1696
Epoch 16/100
4/4 [==============================] - 0s 86ms/step - loss: 0.1444 - val_loss: 0.0613
Epoch 17/100
4/4 [==============================] - 0s 86ms/step - loss: 0.1201 - val_loss: 0.0620
Epoch 18/100
4/4 [==============================] - 0s 87ms/step - loss: 0.1166 - val_loss: 0.1544
Epoch 19/100
4/4 [=====

In [11]:
y_org = y_agg.loc[t_test_start[0]:t_test_end[551],:]

### Calculate R^2

In [12]:
1-np.sum((np.squeeze(y_org)-np.squeeze(preds))**2)/np.sum((y_org-np.mean(y_org))**2)

0   -0.250568
dtype: float64

### A good R^2 is expected to be between 0 to 0.05.